In [ ]:
def gini_impurity(data, target_attr):
    """Berechnet die Gini-Impurity einer Liste basierend auf dem Zielattribut."""
    counts = Counter(row[target_attr] for row in data)
    total = len(data)
    return 1 - sum((count / total) ** 2 for count in counts.values())

def gini_gain(data, split_attr, target_attr):
    """Berechnet den Gini-Gewinn für ein gegebenes Attribut."""
    total_impurity = gini_impurity(data, target_attr)
    values = set(row[split_attr] for row in data)
    subset_impurity = 0

    for value in values:
        subset = [row for row in data if row[split_attr] == value]
        weight = len(subset) / len(data)
        subset_impurity += weight * gini_impurity(subset, target_attr)

    return total_impurity - subset_impurity

def id3(data, attributes, target_attr):
    """Rekursive Implementierung des ID3-Algorithmus basierend auf Gini-Gewinn."""
    # Abbruchbedingung: Wenn alle Zielwerte gleich sind
    target_values = [row[target_attr] for row in data]
    if len(set(target_values)) == 1:
        return target_values[0]

    # Abbruchbedingung: Wenn keine Attribute mehr übrig sind
    if not attributes:
        return Counter(target_values).most_common(1)[0][0]

    # Wähle das Attribut mit dem höchsten Gini-Gewinn
    gains = {attr: gini_gain(data, attr, target_attr) for attr in attributes}
    best_attr = max(gains, key=gains.get)

    tree = {best_attr: {}}
    values = set(row[best_attr] for row in data)

    for value in values:
        subset = [row for row in data if row[best_attr] == value]
        subtree = id3(subset, [attr for attr in attributes if attr != best_attr], target_attr)
        tree[best_attr][value] = subtree

    return tree

def print_tree(tree, indent=""):
    """Formatiert die Ausgabe des Entscheidungsbaums."""
    if isinstance(tree, dict):
        for attr, branches in tree.items():
            for value, subtree in branches.items():
                print(f"{indent}{attr} = {value}:")
                print_tree(subtree, indent + "  ")
    else:
        print(f"{indent}--> {tree}")

def evaluate(tree, sample):
    """Evaluieren eines Datensatzes mit dem Entscheidungsbaum."""
    if not isinstance(tree, dict):
        return tree

    attr = next(iter(tree))
    value = sample.get(attr)
    if value not in tree[attr]:
        print('Unbekannter Wert:', value)
        return None  # Unbekannter Wert
    return evaluate(tree[attr][value], sample)


[Datensatz für die Klassifizierung mit Entscheidungsbäumen](https://www.kaggle.com/datasets/akalyasubramanian/dataset-for-decision-tree-classification) von kaggle.com

In [ ]:
# https://www.kaggle.com/datasets/akalyasubramanian/dataset-for-decision-tree-classification
import kagglehub
import pandas as pd
import math
from collections import Counter

# Kaggle-Datensatz herunterladen
path = kagglehub.dataset_download("akalyasubramanian/dataset-for-decision-tree-classification")
print("Path to dataset files:", path)

# Datensatz laden (angenommen, es ist eine CSV-Datei)
csv_path = path + "/Purchase_new.csv"
df = pd.read_csv(csv_path)

# Datensatz in das passende Format umwandeln
data = df.to_dict(orient="records")
print('Spalten', df.columns)
# Zielattribut bestimmen
target_attr = "Purchase"

# Attribute extrahieren (alle Spalten außer der Zielspalte)
attributes = [col for col in df.columns if col != target_attr]
print('Attribute', attributes)

# ID3-Algorithmus anwenden
decision_tree = id3(data, attributes, target_attr)

# Entscheidungsbaum ausgeben
print("Entscheidungsbaum:")
print_tree(decision_tree)

# Beispiel evaluieren
# ACHTUNG: Im Datensatz sind die YES-Werte bei Holiday leider mit kleinem y geschrieben!!!
sample = {"Discount": "No", "Free Delivery": "Yes", "Holiday": "yes"}
result = evaluate(decision_tree, sample)
print("\nEvaluierung des Beispiels:")
print(f"Beispiel: {sample}")
print(f"Ergebnis: {result}")



100%|██████████| 277/277 [00:00<00:00, 575kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/akalyasubramanian/dataset-for-decision-tree-classification/versions/1
Spalten Index(['Holiday', 'Discount', 'Free Delivery', 'Purchase'], dtype='object')
Attribute ['Holiday', 'Discount', 'Free Delivery']
Entscheidungsbaum:
Discount = Yes:
  Holiday = No:
    --> Yes
  Holiday = yes:
    Free Delivery = Yes:
      --> Yes
    Free Delivery = No:
      --> Yes
Discount = No:
  Free Delivery = Yes:
    Holiday = No:
      --> No
    Holiday = yes:
      --> Yes
  Free Delivery = No:
    Holiday = No:
      --> No
    Holiday = yes:
      --> No

Evaluierung des Beispiels:
Beispiel: {'Discount': 'No', 'Free Delivery': 'Yes', 'Holiday': 'yes'}
Ergebnis: Yes
